# 07. 개인 플레이어 기여도 분석

**기여도 = 예측 골드 / 전체 평균 골드** (1.0 = 평균)

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import platform
import os
import glob
import time
import random
import gc

# ==========================================
# 0. 기본 설정
# ==========================================
def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False

set_korean_font()

# 파일 경로
DATA_PATH = "data/final_features.csv"
TARGET_PUUID_PATH = "data/puuid_challengers_0408.txt"
MODEL_DIR = "models"
OUTPUT_DIR = "output/personal_analysis"  # 결과 저장 경로 설정

ALL_POSITIONS = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
MIN_GAMES = 10

FEATURE_COLS = [
    'xp', 'minionsKilled', 'jungleMinionsKilled',
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills', 'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff',
    'KDA', 'killParticipation', 'damageShare'
]

# 결과 저장 폴더 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================
# 1. 데이터 로드 & 초기화
# ==========================================
def load_target_puuids(filepath):
    if not os.path.exists(filepath): return []
    with open(filepath, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]

def find_latest_model(position):
    search_pattern = os.path.join(MODEL_DIR, f"xgb_gold_predictor_{position}_*.pkl")
    files = glob.glob(search_pattern)
    return max(files, key=os.path.getctime) if files else None

print("📂 데이터 로딩 시작...")

challenger_puuids = load_target_puuids(TARGET_PUUID_PATH)
challenger_set = set(challenger_puuids)

if not challenger_puuids:
    print("❌ PUUID 파일 오류")

print("   데이터 읽기 및 필터링 중...")
# 데이터 로드 시 예외 처리 추가
try:
    df_all = pd.read_csv(DATA_PATH)
    df_challengers = df_all[df_all['puuid'].isin(challenger_set)].copy()
    df_challengers = df_challengers[df_challengers['minute'] > 0]
    del df_all
    gc.collect()
    print(f"   ✅ 챌린저 데이터 확보: {len(df_challengers):,} rows")
except FileNotFoundError:
    print(f"❌ 데이터 파일을 찾을 수 없습니다: {DATA_PATH}")
    df_challengers = pd.DataFrame()

print("   포지션별 평균 골드 계산 중...")
GLOBAL_MEAN_GOLD = {}
if not df_challengers.empty:
    for pos in ALL_POSITIONS:
        df_pos = df_challengers[df_challengers['position'] == pos]
        if not df_pos.empty:
            GLOBAL_MEAN_GOLD[pos] = df_pos.groupby('minute')['totalGold'].mean()

MODELS = {}
print("   ✅ 초기화 완료")


# ==========================================
# 2. 분석 및 시각화 함수 (저장 기능 추가)
# ==========================================
def analyze_player_contribution_lazy(df_player_data, position):
    df_target = df_player_data[df_player_data['position'] == position].copy()
    if len(df_target) == 0: return None

    unique_matches = df_target['matchId'].nunique()
    if unique_matches < MIN_GAMES: return None

    if position not in MODELS:
        print(f"      📥 [{position}] 모델 로딩 중... (최초 1회)")
        model_path = find_latest_model(position)
        if model_path:
            MODELS[position] = joblib.load(model_path)
        else:
            print(f"      ❌ [{position}] 모델 파일 없음")
            return None

    model = MODELS[position]
    global_mean_dict = GLOBAL_MEAN_GOLD.get(position, pd.Series()).to_dict()

    # 예측 수행 (predictedGold 생성)
    X_target = df_target[FEATURE_COLS]
    df_target['predictedGold'] = model.predict(X_target)

    df_target['global_mean'] = df_target['minute'].map(global_mean_dict)
    df_target['global_mean'] = df_target['global_mean'].fillna(1).replace(0, 1)
    df_target['minute_contribution'] = df_target['predictedGold'] / df_target['global_mean']

    match_stats = df_target.groupby(['matchId', 'gameStartDate', 'win']).agg({
        'minute_contribution': 'mean'
    }).reset_index()

    match_stats['gameStartDate'] = pd.to_datetime(match_stats['gameStartDate'])
    match_stats = match_stats.sort_values('gameStartDate').reset_index(drop=True)

    return match_stats, df_target

def plot_contribution_timeline(match_stats, puuid, position, save_path=None):
    fig, ax = plt.subplots(figsize=(12, 5))

    dates = match_stats['gameStartDate']
    y = match_stats['minute_contribution']
    wins = match_stats['win']

    colors = np.where(wins == 1, '#4CAF50', '#F44336')
    ax.scatter(dates, y, c=colors, s=80, alpha=0.8, edgecolors='white', zorder=3)

    x_numeric = np.arange(len(dates))
    slope = 0
    if len(x_numeric) > 1:
        z = np.polyfit(x_numeric, y, 1)
        p = np.poly1d(z)
        slope = z[0]
        trend_color = '#2196F3' if slope > 0 else '#FF9800'
        ax.plot(dates, p(x_numeric), color=trend_color, linestyle='--', linewidth=2)

    ax.axhline(y=1.0, color='black', alpha=0.5, label='챌린저 평균(1.0)')

    win_rate = wins.mean() * 100
    trend_text = "상승(▲)" if slope > 0 else "하락(▼)"

    ax.set_title(f'1. [{position}] 기여도 변화 (판수: {len(match_stats)}, 승률: {win_rate:.1f}%)', fontsize=14, fontweight='bold')
    ax.set_ylabel('기여도 (1.0 = 평균)')
    ax.text(0.02, 0.95, f'추세: {trend_text} ({slope:+.4f})', transform=ax.transAxes,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    ax.grid(True, alpha=0.3)
    plt.tight_layout()

    # 저장 로직
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close() # 메모리 해제
    else:
        plt.show()

    return slope

def plot_gold_curve_comparison(df_player_target, position, save_path=None):
    """실제 골드 vs 예측 골드 vs 챌린저 평균 비교"""

    if position not in GLOBAL_MEAN_GOLD:
        return
    challenger_mean = GLOBAL_MEAN_GOLD[position]

    # 선수 실제 골드 평균
    player_real_mean = df_player_target.groupby('minute')['totalGold'].mean()
    # 선수 예측 골드 평균 (모델이 예측한 값)
    player_pred_mean = df_player_target.groupby('minute')['predictedGold'].mean()

    fig, ax = plt.subplots(figsize=(12, 6))

    max_minute = player_real_mean.index.max()
    valid_minutes = challenger_mean.index[challenger_mean.index <= max_minute]

    # 1. 챌린저 평균 (회색 점선)
    ax.plot(valid_minutes, challenger_mean[valid_minutes],
            color='gray', linestyle='--', linewidth=2, label='챌린저 전체 평균', alpha=0.6)

    # 2. 선수 실제 골드 (주황색 실선)
    ax.plot(player_real_mean.index, player_real_mean.values,
            color='#FF9800', linewidth=3, label='선수 실제 골드')

    # 3. 선수 예측 골드 (파란색 점선)
    ax.plot(player_pred_mean.index, player_pred_mean.values,
            color='#2196F3', linestyle=':', linewidth=2.5, label='모델 예측 골드')

    # 영역 채우기 (실제 vs 평균)
    common_idx = player_real_mean.index.intersection(challenger_mean.index)
    if len(common_idx) > 0:
        p_vals = player_real_mean[common_idx]
        c_vals = challenger_mean[common_idx]

        ax.fill_between(common_idx, p_vals, c_vals,
                        where=(p_vals >= c_vals), interpolate=True, color='#4CAF50', alpha=0.15)
        ax.fill_between(common_idx, p_vals, c_vals,
                        where=(p_vals < c_vals), interpolate=True, color='#F44336', alpha=0.15)

    ax.set_title(f'2. [{position}] 골드 획득량 비교 (실제 vs 예측 vs 챌린저평균)', fontsize=14, fontweight='bold')
    ax.set_xlabel('게임 시간 (분)')
    ax.set_ylabel('골드 획득량')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)

    plt.tight_layout()

    # 저장 로직
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close() # 메모리 해제
    else:
        plt.show()


📂 데이터 로딩 시작...
   데이터 읽기 및 필터링 중...
   ✅ 챌린저 데이터 확보: 1,493,916 rows
   포지션별 평균 골드 계산 중...
   ✅ 초기화 완료


In [12]:
# ==========================================
# 3. 실행 (Main Execution)
# ==========================================

if challenger_puuids and not df_challengers.empty:
    TARGET_PUUID = random.choice(challenger_puuids)
    print("\n" + "="*60)
    print(f"🎲 랜덤 선택된 플레이어: {TARGET_PUUID}")
    print(f"📂 결과 저장 경로: {os.path.abspath(OUTPUT_DIR)}")
    print("="*60)
else:
    TARGET_PUUID = ""
    print("❌ 분석 대상 데이터 또는 PUUID가 없습니다.")

if TARGET_PUUID:
    df_player = df_challengers[df_challengers['puuid'] == TARGET_PUUID].copy()

    if df_player.empty:
        print("❌ 해당 플레이어 데이터 없음")
    else:
        pos_counts = df_player.groupby('position')['matchId'].nunique().sort_values(ascending=False)
        target_positions = [pos for pos, count in pos_counts.items() if count >= MIN_GAMES]

        print(f"📊 분석 대상 포지션: {target_positions}")

        for pos in target_positions:
            print(f"\n🔍 [{pos}] 분석 진입...")
            try:
                result = analyze_player_contribution_lazy(df_player, pos)

                if result is not None:
                    stats, df_pos_data = result

                    # 파일명 생성을 위한 ID 단축 (앞 8자리)
                    short_id = TARGET_PUUID[:8]

                    # 1. 기여도 타임라인 저장
                    path_timeline = os.path.join(OUTPUT_DIR, f"{short_id}_{pos}_timeline.png")
                    plot_contribution_timeline(stats, TARGET_PUUID, pos, save_path=path_timeline)
                    print(f"   💾 타임라인 그래프 저장됨: {path_timeline}")

                    # 2. 골드 커브 비교 저장
                    path_gold = os.path.join(OUTPUT_DIR, f"{short_id}_{pos}_gold_curve.png")
                    plot_gold_curve_comparison(df_pos_data, pos, save_path=path_gold)
                    print(f"   💾 골드커브 그래프 저장됨: {path_gold}")

                else:
                    print(f"   결과: 분석 요건 미충족 (데이터 부족 등)")
            except Exception as e:
                print(f"   ⚠️ 에러 발생: {e}")
                # import traceback
                # traceback.print_exc()

print("\n✅ 분석 및 저장 종료")


🎲 랜덤 선택된 플레이어: lGmb9idWN6trpV7av9MLMEsO7TGGtNgOBOS43LAx45oRLiCaaowIRuCgAZ84mQLZx7T1jJeeS-7_lA
📂 결과 저장 경로: /Users/jaewon/Desktop/lol_data_analysis/output/personal_analysis
📊 분석 대상 포지션: ['TOP']

🔍 [TOP] 분석 진입...
   💾 타임라인 그래프 저장됨: output/personal_analysis/lGmb9idW_TOP_timeline.png
   💾 골드커브 그래프 저장됨: output/personal_analysis/lGmb9idW_TOP_gold_curve.png

✅ 분석 및 저장 종료
